In [ ]:
import xarray as xr
import numpy as np
import rasterio
import uav
import os

from tqdm import tqdm
from glob import glob


output = r'D:\sentinel2-acolite-prod\jupyters\Mar\results\shallow_and_deep_water\raster'
os.makedirs(output, exist_ok = True)

variables = ['chl_re_mishra', 'qaa_v6_KPAR_Lee', 'SPM_Nechad2016_665',
             'p3qaa_zSD', 'qaa_v6_Kd_490']
methods = [('mean', np.nanmean), ('median', np.nanmedian), ('std', np.nanstd)]

with xr.open_dataset(r"D:\NRT-Sentinel2\outputs\FRP_Lagoon\section_1\FRP\20170105_S2A\20170105_S2A_FRP_22JGQ_L2W_clipped.nc") as src:
    new_ds = src.drop_vars(list(src.keys()))

    ndwi_filename = r'D:\sentinel2-acolite-prod\jupyters\Mar\masks\ndwi.tif'

    with rasterio.open(ndwi_filename) as src:
        is_water = src.read() > 0.1
        water_mask_tif = uav.GeoreferenceData(is_water, src.transform)

    with rasterio.open(r'D:\sentinel2-acolite-prod\jupyters\Mar\base_georeference_reduced.tif') as reference:
        transform = reference.transform
        height, width = reference.read(1).shape


    # We calculate anual raster for each variable masking by NDWI
    for method_name, method in methods:
        for year in ['2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']:
            print(f'START {year}')
            
            data_by_vars = {}
            for filename in tqdm(glob(rf'D:\NRT-Sentinel2\outputs\FRP_Lagoon\section_1\FRP\{year}*\*L2W_clipped.nc')):
                with xr.open_dataset(filename) as src:
                    xs, ys = np.meshgrid(src.x, src.y)
                    rows, cols = water_mask_tif.get_row_col_by_lon_lat(xs, ys)
                    in_shape = water_mask_tif.get_in_shape_mask(rows, cols)
                    is_water_mask = water_mask_tif.get_data_by_row_col(rows[in_shape], cols[in_shape])[0].reshape((height, width))
                    
                    for variable in variables:
                        if variable in src:
                            data = src[variable].values
                            data[~is_water_mask] = np.nan
                            data_by_vars.setdefault(variable, []).append(data)

            for variable, data in data_by_vars.items():
                data_by_vars[variable] = method(np.array(data), axis = 0)
                new_ds[f'{variable}_{year}'] = (('y', 'x'), method(np.array(data), axis = 0))

            print(f'END {year}')

        print(f'START ALL YEARS')
        data_by_vars = {}
        for filename in tqdm(glob(rf'D:\NRT-Sentinel2\outputs\FRP_Lagoon\section_1\FRP\*\*L2W_clipped.nc')):
            with xr.open_dataset(filename) as src:
                xs, ys = np.meshgrid(src.x, src.y)
                rows, cols = water_mask_tif.get_row_col_by_lon_lat(xs, ys)
                in_shape = water_mask_tif.get_in_shape_mask(rows, cols)
                is_water_mask = water_mask_tif.get_data_by_row_col(rows[in_shape], cols[in_shape])[0].reshape((height, width))
                
                for variable in variables:
                    if variable in src:
                        data = src[variable].values
                        data[~is_water_mask] = np.nan
                        data_by_vars.setdefault(variable, []).append(data)

        for variable, data in data_by_vars.items():
            data_by_vars[variable] = method(np.array(data), axis = 0)
            new_ds[f'{variable}_all_years'] = (('y', 'x'), method(np.array(data), axis = 0))
        print(f'END ALL YEARS')

        new_ds.to_netcdf(rf'{output}\{method_name}.nc')

In [ ]:
import xarray as xr
import os

filenames = [
    r"D:\sentinel2-acolite-prod\jupyters\Mar\results\shallow_and_deep_water\raster\mean.nc",
    r"D:\sentinel2-acolite-prod\jupyters\Mar\results\shallow_and_deep_water\raster\median.nc",
    r"D:\sentinel2-acolite-prod\jupyters\Mar\results\shallow_and_deep_water\raster\std.nc",
]

masks_by_code = {
    '2017' : 'deep water mask 2017',
    '2018' : 'deep water mask 2018',
    '2019' : 'deep water mask 2019',
    '2020' : 'deep water mask 2020',
    '2021' : 'deep water mask 2021',
    '2022' : 'deep water mask 2022',
    '2023' : 'deep water mask 2023',
    '2024' : 'deep water mask 2024',
    'years' : 'deep water mask all_years',
}

# Apply deep water mask for each yearly variable
with xr.open_dataset(r"D:\sentinel2-acolite-prod\jupyters\Mar\masks\deep_water.nc") as deep_file:
    for filename in filenames:
        name = os.path.basename(filename)
        
        # mask deepwater
        outname = rf'D:\sentinel2-acolite-prod\jupyters\Mar\results\shallow_water\{name}'
        with xr.open_dataset(filename) as src:
            for var in src:
                code = var.split('_')[-1]
                deep_water_mask = deep_file[masks_by_code[code]].values
                data = src[var].values
                data[deep_water_mask] = np.nan
                src[var] = (('y', 'x'), data)
        
            src.to_netcdf(outname)

        # mask inverse of deepwater
        outname = rf'D:\sentinel2-acolite-prod\jupyters\Mar\results\deep_water\{name}'
        with xr.open_dataset(filename) as src:
            for var in src:
                code = var.split('_')[-1]
                deep_water_mask = deep_file[masks_by_code[code]].values
                data = src[var].values
                data[~deep_water_mask] = np.nan
                src[var] = (('y', 'x'), data)
        
            src.to_netcdf(outname)